# Librerias

In [13]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import signal
import os


# Declaracion de constantes 

In [14]:
F_MIN = 3
F_MAX = 20
F_MUESTREO = 128
RUTA_DATOS_PROCESADOS = '../DatosProcesados/Frecuencia'
CARPETAS = ['Rojo', 'Azul', 'Verde', 'Morado', 'Baseline']

# Declaracion de Funciones 

In [15]:
def plot_datos(datos):
    
    plt.figure()
    plt.plot(datos[:, 0], label = 'EEG.O1')
    plt.plot(datos[:, 1], label = 'EEG.O2')
    plt.legend()
    plt.title('Raw Data')
    plt.xlabel('Tiempo')
    plt.ylabel('uV')
    
def plot_psd_semilog(datos):
    
    f1, psd_O1 = signal.periodogram(datos[:, 0], F_MUESTREO)
    f2, psd_O2 = signal.periodogram(datos[:, 1], F_MUESTREO)
    plt.semilogy(f1, psd_O1, label = 'EEG.O1')
    plt.semilogy(f1, psd_O2, label = 'EEG.O2')
    plt.ylim([1e-7, 1e2])
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.legend()

    
def plot_psd(datos):
    
    plt.figure()
    f1, psd_O1 = signal.periodogram(datos[:, 0], F_MUESTREO)
    f2, psd_O2 = signal.periodogram(datos[:, 1], F_MUESTREO)
    plt.plot(f1, psd_O1, label = 'EEG.O1')
    plt.plot(f1, psd_O2, label = 'EEG.O2')
    plt.ylim([0, 30])
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD')
    plt.legend()
    

def filtro_pasa_banda(datos, f_min, f_max, f_muestreo, orden=5):
    nyq = 0.5 * f_muestreo
    f_min = f_min / nyq
    f_max = f_max / nyq
    b, a = signal.butter(orden, [f_min, f_max], btype='band')
    y = signal.lfilter(b, a, datos)
    return y

## Graficas de Datos (Sin filtro)

In [ ]:
for carpeta in CARPETAS:
    archivos = [e for e in os.listdir(RUTA_DATOS_PROCESADOS+'/'+carpeta) if e.endswith('.csv') ]
    
    for indice, archivo in enumerate(archivos):
        ruta_origen = f'{RUTA_DATOS_PROCESADOS}/{carpeta}/{archivo}'
        df = pd.read_csv(ruta_origen)
        datos = df[['EEG.O1', 'EEG.O2']].to_numpy()
        plot_psd(datos)
        plt.savefig(f'{RUTA_DATOS_PROCESADOS}/{carpeta}/sinfiltro/psd/{archivo[:-4]}')

In [ ]:
!rm ../DatosProcesados/Frecuencia/Rojo/sinfiltro/raw/*.png